### # Estimate calibration factor alpha from 3D beads data (zyx)

This file contains a basic pipeline for QP retrieval from brightfield stacks
* 4D image stack loading
* 4D stack preprocessing
* processing parameters definition
* phase calculation
* display of the results


This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License, with this program.  If not, see <http://www.gnu.org/licenses/>.


# load modules

In [1]:
import numpy as np
import os
#import napari
#from utils.io import loadData, writeData, loadDataOld
from utils.cropXY import cropXY
from utils.cropCoregMask import cropCoregMask
from utils.phase_structure import phase_structure
from utils.getQP import getQP
#from utils.plotStack import plotStack

In [2]:
# modify QP retrieval to iterate gamma output
import numpy as np
from numpy import dot, pi, sin, cos, angle, logical_and, logical_or
import numpy.fft as fft
import math
import time
from utils.getMirroredStack import getMirroredStack
from utils.map3D import map3D
from skimage import io
import numpy as np
from numpy import dot
from copy import copy  
import matplotlib.pyplot as plt
import scipy 
import tifffile as tf

def getMirroredStack(stack=None,s=None,*args,**kwargs):

    Nx,Ny,Nz=stack.shape
    
    if Nx != Ny:                     # verify that the stack is square
        stack=cropXY(stack)          # if not, crop it
    
    # compute real space
    x=np.linspace(dot(- Nx,s.optics_dx) / 2,dot(Nx,s.optics_dx) / 2,Nx)
    z=np.linspace(dot(- Nz,s.optics_dz) / 2,dot(Nz,s.optics_dz) / 2,Nz)
    # mirror z-stack
    temp=copy(stack)
    if s.proc_mirrorZ:
        t=copy(temp)
        t = np.append(t,temp[:, :, ::-1], 2)
        temp=copy(t)
        kz=np.multiply(dot(2,np.pi) / (max(z) - min(z)),np.linspace(- Nz / 2,(Nz - 1) / 2,dot(2,Nz)))
    else:
        if np.mod(Nz,2):
            kz=np.multiply(dot(2,np.pi) / (max(z) - min(z)),np.linspace(- Nz / 2,Nz / 2,Nz))
        else:
            kz=np.multiply(dot(2,np.pi) / (max(z) - min(z)),np.linspace(- Nz / 2,Nz / 2 - 1,Nz))
            
    # mirror x dim
    if s.proc_mirrorX:
        t=copy(temp)
        t = np.append(t,temp[::-1,:, :],0)
        t[int(len(t)/2):,:,:] = t[:int(len(t)/2):-1,:,:]
    
        temp=copy(t)
        kx=np.multiply(dot(2,np.pi) / (max(x) - min(x)),np.linspace(- Nx / 2,(Nx - 1) / 2,dot(2,Nx)))
    else:
        if np.mod(Nz,2):
            kx=np.multiply(dot(2,np.pi) / (max(x) - min(x)),np.linspace(- Nx / 2,Nx / 2,Nx))
        else:
            kx=np.multiply(dot(2,np.pi) / (max(x) - min(x)),np.linspace(- Nx / 2,Nx / 2 - 1,Nx))
    stackM = temp
    
    return kx,kz,stackM


def getQP(stack=None,struct=None,mask=None, log = False):
    print(f"struct.optics_alpha: {struct.optics_alpha}")
    #print(f"stack.shape {stack.shape}")
    # mirror the data and compute adequate Fourier space grid
    kx,kz,stackM = getMirroredStack(stack,struct)
    #print(f"stackM.shape {stackM.shape}")
    if mask is None: # if no mask is provided
        # compute usefull stuff
        th=math.asin(struct.optics_NA / struct.optics_n)
        th_ill=math.asin(struct.optics_NA_ill / struct.optics_n)
        k0max = dot(dot(struct.optics_n,2),pi) / (struct.optics_wv - struct.optics_dlambda / 2) 
        k0min = dot(dot(struct.optics_n,2),pi) / (struct.optics_wv + struct.optics_dlambda / 2) 
        
        # compute Fourier space grid and the phase mask
        Kx,Kz=np.meshgrid(kx,kz)
        if struct.optics_kzT is None:
            mask2D = Kz >= np.dot(k0max,(1 - cos(th_ill)))
        else:
            mask2D = Kz >= struct.optics_kzT


        if struct.proc_applyFourierMask:  #  => compute the CTF mask for extra denoising
            # CTF theory
            maskCTF = logical_and(logical_and(logical_and(
                ((Kx - dot(k0max,sin(th_ill))) ** 2 + (Kz - dot(k0max,cos(th_ill))) ** 2) <= k0max ** 2, \
                    ((Kx + dot(k0min,sin(th_ill))) ** 2 + (Kz - k0min) ** 2) >= k0min ** 2), Kx >= 0), \
                    Kz < dot(k0max,(1 - cos(th))))            
            maskCTF = logical_or(maskCTF,maskCTF[:,::-1])
            mask2D = np.asanyarray(logical_and(mask2D,maskCTF), dtype=int)
        # since we assume a circular symetric CTF, we expand the 2Dmask in 3D
        mask=map3D(mask2D)
        #np.save("data\mask3D.npy", mask)

    # Cross-Spectral Density calculation
    if log:
        print("Shifting FFT..")
    Ik=fft.fftshift(fft.fftn(fft.fftshift(stackM)))
    #print(f"stackM.shape {stackM.shape}")
    #print(f"Ik.shape {Ik.shape}")
    #print(f"mask.shape {mask.shape}")
    Gamma=np.multiply(Ik,mask)          # cross-spectral density
    if log:
        print("Inverse FFT..")
    csd=fft.ifftshift(fft.ifftn(fft.ifftshift(Gamma)))
    csd = csd[:stack.shape[0], :stack.shape[1], :stack.shape[2]]   # remove the mirrored input
    
    QP = angle(csd + np.mean(np.ravel(stack)) / struct.optics_alpha)

    #return QP,mask,Gamma
    return QP

def writeData(QP, path, filename, s): 
    outputImageFileName = os.path.join(path, f"QP_{filename}")    
    if len(QP.shape) == 4:
        QP = np.transpose(QP, (2,1,0,3))
        print("4d")
        axes = 'ZYXT'
    else: 
        QP = np.transpose(QP, (2,1,0))
        print("3d")
        axes = "ZXY"
    print(QP.shape)
    tf.imwrite(
        outputImageFileName,
        QP,
        resolution=(0.107, 0.107),
        metadata={ 
            'spacing': s.optics_dz,
            'unit': 'um',
            'finterval': 1,
            'axes': axes
        })

    print(f"writing QP stack {outputImageFileName} finished.")

# load data

In [25]:
#%% 3D image stack loading
file = r"C:\Users\mengelhardt\data\local\2022-11-25\0_50_um_stack_1\rcrop_0_50_um_stack_1_MMStack.ome-1.tif"
QP_theo = 2.81 # theoretical maximum phase value
#z = 4
#t = 50
#stack, fname = loadData(file,"tif", z,t) # loadData also takes filepath as input
stack = io.imread(file)
# flag if you want to explore the data in napari
PLOT_FLAG = True 
print(stack.shape)

(151, 491, 413)


## assign dimensions accordingly
permute if necessary (order should be x,y,z,t) 

In [26]:
Nz,Ny,Nx = stack.shape
print(stack.shape)

(151, 491, 413)


## permute input stack if necessary

In [27]:
NEW_ORDER = [1,2,0]
stack = np.transpose(stack, NEW_ORDER)

Nx,Ny,Nz = stack.shape
print(stack.shape)

(491, 413, 151)


# 3D stack Preprocess stack

In [28]:
if Nz == 8:                                 # i.e. multiplane data: remove the coregistration mask
    stack=cropCoregMask(stack)
    TEMP_NPIX = 100                         #  variable was undefined in mat file, set accordingly         
    stack=cropXY(stack,TEMP_NPIX - 4)       #  extra safety crop 
else:
    stack=cropXY(stack)
Nx,Ny,Nz = stack.shape
print(stack.shape)

Cropping..
Cropping finished
(413, 413, 151)


# Phase retrieval
## define optics and processing parameters

In [29]:
s=phase_structure()

s.optics_kzT = 0.01                 # Axial cutoff frequency
#s.optics_kzT = 1.3 * (np.pi/0.455)*(1-np.sqrt(1-(0.58)**2))
# if set to [], use the theoretical value
s.proc_mirrorX = False             # mirror the input stack along X 
s.proc_mirrorZ = True               # mirror the input stack along Z
s.proc_applyFourierMask = True
# set experimental parameters
if Nz == 4:              # i.e. MultiPlane data
    s.optics_dz = 0.72               # [um]
else:
    s.optics_dz = 0.1               # sampling beads
    
phase_structure.summarise(s)

Phase structure: 
_________________
s.optics_dx = 0.1067 	 	 s.optics_wv = 0.58
s.optics_dz = 0.1 	 	 s.optics_dlambda = 0.075
s.optics_NA = 1.3 	 	 s.optics_alpha = 4.21
s.optics_NA_ill = 0.55 	 	 s.optics_kzT = 0.01
s.optics_n = 1.406
Processing paramters: 
_________________
s.proc_mirrorX = False 	 	 s.proc_mirrorZ = True
s.proc_applyFourierMask = True


# compute the phase and optimise iteratively
* if you have a dense sample ( and you are sure that in an image tiled up in 9 equal subimages every part will have a bead) then choose QP_optim_tiled
Target values for: 
* 0.2um: 0.137 (correction factor for PSF size) -> phase: 0.189 
* 0.5um: 1.405
* 0.75um: 2.1075
* 1.00um: 2.81


In [30]:
init_xa, init_xb = 2.5, 4.75

def QP_optim(alpha):
    s.optics_alpha = alpha 
    return np.abs(np.amax(getQP(stack[:,:,:],s)) - QP_theo)


def QP_optim_tiled(alpha):
    s.optics_alpha = alpha 
    rows = np.linspace(0, stack.shape[0], 3).astype(int)
    columns = np.linspace(0, stack.shape[0], 3).astype(int)
    MAXS = []
    for r in range(len(rows)-1):
        for c in range(len(columns)-1):
            print(f"Processing rows {rows[r]}:{rows[r+1]} & cols {columns[c]}:{columns[c+1]}")
            MAXS.append(np.amax(getQP(stack[rows[r]:rows[r+1],columns[c]:columns[c+1],:],s)))
    MAXS = np.sort(np.array(MAXS))
    MAXS = MAXS[1:-1] # filter for tp and bottom outlier 
    return np.abs(np.mean(MAXS) - QP_theo)


In [31]:
OPTIM = scipy.optimize.minimize_scalar(QP_optim_tiled, method='bounded', bounds=(init_xa, init_xb))
print(f"-----------------------------------\n Optimisation finished after {OPTIM.nit} iterations.\n Alpha: {OPTIM.x}")

Processing rows 0:206 & cols 0:206
struct.optics_alpha: 3.3594235253127365
Processing rows 0:206 & cols 206:413
struct.optics_alpha: 3.3594235253127365
Cropping..
Cropping finished
Processing rows 206:413 & cols 0:206
struct.optics_alpha: 3.3594235253127365
Cropping..
Cropping finished


ValueError: operands could not be broadcast together with shapes (206,206,302) (207,207,302) 

In [ ]:
outputTransformFileName = file + ".QP_optim.npy"
np.save(outputTransformFileName, OPTIM)

## run with optimised alpha value

In [ ]:
s=phase_structure()

s.optics_kzT = 0.01                 # Axial cutoff frequency
#s.optics_kzT = 1.3 * (np.pi/0.455)*(1-np.sqrt(1-(0.58)**2))
# if set to [], use the theoretical value
s.proc_mirrorX = False             # mirror the input stack along X 
s.proc_mirrorZ = True               # mirror the input stack along Z
s.proc_applyFourierMask = True
# set experimental parameters
if Nz == 4:              # i.e. MultiPlane data
    s.optics_dz = 0.72               # [um]
else:
    s.optics_dz = 0.1               # typical sampling for fixed cells
    
s.optics_alpha = OPTIM.x
phase_structure.summarise(s)

In [ ]:
phase = getQP(stack,s)

# display phase and input stack next to each other

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0):
    fig = plt.figure(figsize=(20, 20))
    plt.subplot(121)
    plt.imshow(stack[:, :, z], cmap='gray')
    plt.colorbar(label="[e-/ADU]", orientation="vertical", fraction=0.046, pad=0.04)
    plt.title("input stack")
    
    plt.subplot(122)
    plt.imshow(phase[:, :, z], cmap='gray')
    plt.title("QP")
    plt.colorbar(label="[rad]", orientation="vertical", fraction=0.046, pad=0.04)

    plt.tight_layout()

    fig.canvas.flush_events()
    

interact(update, z= widgets.IntSlider(value=1, min=0, max=stack.shape[2]-1, step=1, description="Select Z", continuous_update=True))

# write QP to same folder as input with "QP_" name qualifier 

In [ ]:
separator = "\\"
parse = file.split(separator)
parse[-1] = "QP_" + parse[-1]
output_path = separator.join(parse[:-1])+separator
file = parse[-1]
writeData(phase, output_path, file, s)

# Create Maximum intensity projection

In [ ]:
MIP_phase = np.max(phase, axis=2)
fig = plt.figure(figsize=(20, 20))
plt.title(f"Alpha: {s.optics_alpha}")
io.imshow(MIP_phase)
io.imsave(os.path.join(output_path, "MIP_"+file), MIP_phase)



In [16]:
2.81*0.75

2.1075

In [17]:
1.59* (0.2/0.863) + 1.33* ((0.863-0.2)/0.863)

1.3902549246813443

In [22]:
1.3902549246813443 *(2*np.pi/640)*200

2.729759161247464